# DATA ANONYMIZATION

## Introduction

This notebook show how to use the anonymization feature with an small example. We will start by setting up the notebook, then we will create our dummy dataset and its metadata, and finally we will `model` and `sample` the data, checking the diferencies in both the data and the internal state of the objects.

## Notebook preparation

In [1]:
import json

import numpy as np
import pandas as pd
import rdt

from sdv import SDV

## Creating dataset and metadata

We are going to create a dataset of a single table containing three different columns : `primary_key`, `name` and `credit_card_number` and two different metadata, one that does use anonymization, and the other that it doesn't.

In [23]:
# Generating data for table.
table_data = pd.DataFrame([
    {
        'index': 1,
        'name': 'Bill',
        'credit_card_number': '1111222233334444'
    },
    {
        'index': 2,
        'name': 'Jeff',
        'credit_card_number': '0000000000000000'
    },
    {
        'index': 2,
        'name': 'Warren',
        'credit_card_number': '2222222222222222'
    },
    {
        'index': 2,
        'name': 'Bill',
        'credit_card_number': '9999999999999999'
    },
    {
        'index': 2,
        'name': 'Jeff',
        'credit_card_number': '8888888888888888'
    },
])

# Storing as CSV
table_data.to_csv('table.csv', index=False, header=True)

Now we are going to generate the metadata. There are, a part from the anonymization parameters, two major differences with the other example metadata:

In [24]:
normal_table_name = 'normal'
normal_table_metadata = {
    'fields': [
        {
            'name': 'name', 
            'type': 'categorical',
        },
        {
            'name': 'credit_card_number',
            'type': 'categorical',
        },
        {
            'name': 'index',
            'subtype': 'integer', 
            'type': 'number', 
            'regex': '^[0-9]{10}$'
        },

    ],
    'headers': True,
    'name': normal_table_name,
    'path': 'table.csv',
    'primary_key': 'index',
    'use': True
}
normal_table_metadata

{'fields': [{'name': 'name', 'type': 'categorical'},
  {'name': 'credit_card_number', 'type': 'categorical'},
  {'name': 'index',
   'subtype': 'integer',
   'type': 'number',
   'regex': '^[0-9]{10}$'}],
 'headers': True,
 'name': 'normal',
 'path': 'table.csv',
 'primary_key': 'index',
 'use': True}

In [25]:
anon_table_name = 'anon'
anon_table_metadata = {
    'fields': [
        {
            'name': 'name', 
            'type': 'categorical',
            'pii': True,
            'pii_category': 'first_name'
        },
        {
            'name': 'credit_card_number', 
            'type': 'categorical',
            'pii': True,
            'pii_category': ['credit_card_number', 'visa']
        },
        {
            'name': 'index', 
            'subtype': 'integer', 
            'type': 'number', 
            'regex': '^[0-9]{10}$'
        },

    ],
    'headers': True,
    'name': anon_table_name,
    'path': 'table.csv',
    'primary_key': 'index',
    'use': True
}
anon_table_metadata

{'fields': [{'name': 'name',
   'type': 'categorical',
   'pii': True,
   'pii_category': 'first_name'},
  {'name': 'credit_card_number',
   'type': 'categorical',
   'pii': True,
   'pii_category': ['credit_card_number', 'visa']},
  {'name': 'index',
   'subtype': 'integer',
   'type': 'number',
   'regex': '^[0-9]{10}$'}],
 'headers': True,
 'name': 'anon',
 'path': 'table.csv',
 'primary_key': 'index',
 'use': True}

In [26]:
metadata = {
    'path': '',
    'tables': [
        anon_table_metadata,
        normal_table_metadata
    ]
}

metadata_filename = 'metadata.json'
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f)

Now we have all that we needed in order to model and sample our example dataset, that is:

- A table of data stored as `table.csv` file
- Two table metadata, both **using to the same table** , but only **one of them anonymizing data**, and each of them using a different name.
- A full metadata specification, including the table metadata mentioned above, stored as `metadata.json`


# Modelling the dataset


Now that we have prepared our data and metadata files is time to model and sample them. To do so, we will:

1. Create an instance of `SDV`.
2. Model the database calling its `fit` method.
3. Generate samples for each table.

In [27]:
from sdv import SDV

sdv = SDV(metadata_filename)
sdv.fit()

## Sample and compare results

Now we are ready to samnple some data.

We will sample data from tables `anon` and `normal` that are originated from the same exact dataframe as we have confirmed before. The behavior that we are expecting is that on the anonymized table, unique values on the columns `credit_card_number` and `name` are not a subset of the unique values of the same columns in the original data table

In [29]:
anon_sampled = sdv.sample_rows('anon', 10)
anon_sampled['anon']

,name,credit_card_number,index
0,Trevor,4622234016568086,0
1,Daniel,4362226958186723,1
2,Daniel,4362226958186723,2
3,Daniel,4685715615756449,3
4,Trevor,4627651130447035,4
5,Daniel,4997383586087675,5
6,Trevor,4997383586087675,6
7,Trevor,4685715615756449,7
8,Jessica,4622234016568086,8
9,Trevor,4685715615756449,9


Here we can see, that the `name` and `credit_card_number` have different values that on the original data, for exemple, in the names column, unique values have changed from `['Bill', 'Jeff', 'Warren']` to `['Jodi', 'David', 'Darrell']`. (Please note that this concrete values are from this execution, and running this notebook again, may yield different results)

On the `credit_card_number` the difference is even more noticeable as they don't have keep the same format. This is not an issue as this data will be transfomed before being passed to the `Modeler` and the transformation for categorical values into numeric should yield close enough results.



In [30]:
normal_sampled = sdv.sample_rows('normal', 10)
normal_sampled['normal']

,name,credit_card_number,index
0,Bill,0000000000000000,0
1,Jeff,0000000000000000,1
2,Jeff,2222222222222222,2
3,Jeff,8888888888888888,3
4,Bill,0000000000000000,4
5,Bill,1111222233334444,5
6,Jeff,0000000000000000,6
7,Bill,9999999999999999,7
8,Jeff,9999999999999999,8
9,Warren,8888888888888888,9
